### Calcular para cada tickets con cuanto tiempo de antelación se compró

In [80]:
sc.textFile('../../Data/spark/coupon150720.csv').first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [81]:
from time import strptime
from time import mktime

In [8]:
d1 = strptime('150904' , '%y%m%d')

In [9]:
d2 = strptime('150623', '%y%m%d')

In [11]:
(mktime(d1)-mktime(d2))/(3600*24)

73.0

In [82]:
sc.textFile('../../Data/spark/transm150720.csv').first()

u'79062005698500,TKTT,30,150719,FR,0.0,EUR,T,T,141025,PARA127A8,0.0,EUR,   ,EX,,150719,0.0'

In [ ]:
# Sacamos una tabla con TCN,$,Dif. en días fecha compra - fecha salida (min mktime)

In [106]:
def getTCNDineroFechaSalida(l):
    elems = l.split(",")
    tcn = elems[0]
    dinero = elems[6]
    salida = elems[12]
    return (tcn,salida)

In [107]:
cs = sc.textFile('../../Data/spark/coupon150720.csv').map(getTCNDineroFechaSalida)
cs.take(3)

[(u'79062005698500', u'150904'),
 (u'79062005698500', u'150905'),
 (u'79062005924069', u'150721')]

In [108]:
def getTCNFechaCompra(l):
    elems = l.split(",")
    tcn = elems[0]
    venta = elems[9]
    return (tcn,venta)

In [109]:
ts = sc.textFile('../../Data/spark/transm150720.csv').map(getTCNFechaCompra)
ts.take(3)

[(u'79062005698500', u'141025'),
 (u'79062005924069', u'150719'),
 (u'79065668570385', u'150626')]

In [110]:
fts = ts.filter(lambda x: x[1] != '')
fcs = cs.filter(lambda x: x[2] != '') 

In [111]:
def fechaMenor(ds):
    segs = [mktime(strptime(d, '%y%m%d')) for d in ds]
    #print [segs for segs in ds]
    return min(segs)

In [112]:
fts.cogroup(fcs).cache()

PythonRDD[311] at RDD at PythonRDD.scala:43

In [99]:
fts.cogroup(fcs).mapValues(lambda x: ((fechaMenor(x[1]),fechaMenor(x[0]))).take(1))

PythonRDD[283] at RDD at PythonRDD.scala:43

In [114]:
fts.cogroup(fcs).mapValues(lambda x: (fechaMenor(x[1])-fechaMenor(x[0]))).take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 64.0 failed 1 times, most recent failure: Lost task 2.0 in stage 64.0 (TID 151, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-110-eeba95881fac>", line 2, in <lambda>
IndexError: tuple index out of range

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonR

In [55]:
r = fts.cogroup(fcs).mapValues(lambda x: (fechaMenor(x[1])-fechaMenor(x[0])))

In [56]:
r.take(2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 1 times, most recent failure: Lost task 0.0 in stage 25.0 (TID 57, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1295, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 1907, in <lambda>
    map_values_fn = lambda kv: (kv[0], f(kv[1]))
  File "<ipython-input-

In [ ]:
r.mapValues(lambda dif: dif/(3600*24)).map(lambda x: (int(x[1]),1)).reduceByKey(lambda x,y: x+y)